In [1]:
!jupyter kernelspec list

Available kernels:
  pyspark        C:\Anaconda3\envs\aiml\share\jupyter\kernels\PySpark
  pysparkauto    C:\Anaconda3\envs\aiml\share\jupyter\kernels\PySparkAuto
  python3        C:\Anaconda3\envs\aiml\share\jupyter\kernels\python3


In [3]:
%pycat C:\Anaconda3\envs\aiml\share\jupyter\kernels\PySpark\kernel.json
    

In [4]:
from pyspark.sql import *
import pyspark.sql.functions as F 


In [5]:
spark = SparkSession.builder.appName("basic").getOrCreate()

In [6]:
spark

In [11]:
len(dir(DataFrame)), len(dir(Column)), len(dir(F))
file = 'data/spark/Cartier+for+WinnersCurse.csv'
sdf = spark.read.format('csv').option("header", "true")\
                .option("inferSchema", "true").load(file)
sdf.show()

In [15]:
sdf.count()
sdf.rdd.getNumPartitions()

#ops
sdf.columns
sdf.dtypes
sdf.printSchema()

1

In [24]:
#filter
sdf.filter("bidderrate == 5")   #.show()
#or
sdf.filter(sdf.bidderrate == 5) #.show()
#or 
sdf.filter(sdf['bidderrate'] == 5) #.show()
#or 
sdf.filter(F.col('bidderrate') == 5)  #.show()  #This version

#display rows where bidder is "adammurry" or carloss8055
sdf.filter(F.col('bidder').isin("adammurry" , 'carloss8055') ).show()

+----------+------+-----------+-----------+----------+-------+-------+
| auctionid|   bid|    bidtime|     bidder|bidderrate|openbid|  price|
+----------+------+-----------+-----------+----------+-------+-------+
|1638843936|1500.0|5.226377315|carloss8055|         5|  500.0| 1625.0|
|1638843936|1550.0|6.892916667|carloss8055|         5|  500.0| 1625.0|
|1638843936|1625.0|6.893113426|carloss8055|         5|  500.0| 1625.0|
|1638844464| 450.0|6.736319444|  adammurry|         5|  300.0|  740.0|
|1638844464| 500.0|6.736469907|  adammurry|         5|  300.0|  740.0|
|1638844464| 700.0|6.995763889|  adammurry|         5|  300.0|  740.0|
|1638844464| 740.0| 6.99974537|  adammurry|         5|  300.0|  740.0|
|1640550476| 260.0|6.917569444|carloss8055|         5|    1.0|  300.0|
|1640550476| 280.0|6.980590278|carloss8055|         5|    1.0|  300.0|
|1641628327|1650.0|6.951226852|carloss8055|         5|  500.0|1894.78|
|1641628327|1700.0| 6.95150463|carloss8055|         5|  500.0|1894.78|
|16416

In [29]:
#Select - Transformations vs actions
sdf.select("auctionid")\
    .distinct()\
    .count()

97

In [32]:
sdf.select("auctionid")\
    .distinct().explain(extended=True)

== Parsed Logical Plan ==
Deduplicate [auctionid#10]
+- Project [auctionid#10]
   +- Relation[auctionid#10,bid#11,bidtime#12,bidder#13,bidderrate#14,openbid#15,price#16] csv

== Analyzed Logical Plan ==
auctionid: int
Deduplicate [auctionid#10]
+- Project [auctionid#10]
   +- Relation[auctionid#10,bid#11,bidtime#12,bidder#13,bidderrate#14,openbid#15,price#16] csv

== Optimized Logical Plan ==
Aggregate [auctionid#10], [auctionid#10]
+- Project [auctionid#10]
   +- Relation[auctionid#10,bid#11,bidtime#12,bidder#13,bidderrate#14,openbid#15,price#16] csv

== Physical Plan ==
*(2) HashAggregate(keys=[auctionid#10], functions=[], output=[auctionid#10])
+- Exchange hashpartitioning(auctionid#10, 200)
   +- *(1) HashAggregate(keys=[auctionid#10], functions=[], output=[auctionid#10])
      +- *(1) FileScan csv [auctionid#10] Batched: false, Format: CSV, Location: InMemoryFileIndex[file:/D:/handson/Module1/B2_24_25Sept/data/spark/Cartier+for+WinnersCurse.csv], PartitionFilters: [], PushedFilter

In [36]:
#creation of new column
df3 = sdf.withColumn("new_price", F.col("price") * 2)
#full table aggregation
sdf.agg(F.max("bid"), F.count("bidder")).show()
#per some column eg auctionid 
sdf.groupBy('auctionid').agg(F.sum('bid').alias("bids"), F.count('bidder').alias('cb'))\
   .select("auctionid",'bids', 'cb').show()

+--------+-------------+
|max(bid)|count(bidder)|
+--------+-------------+
|  5400.0|         1348|
+--------+-------------+

+----------+------------------+---+
| auctionid|              bids| cb|
+----------+------------------+---+
|1642005995|           6750.58| 18|
|1647320738|           4330.87| 12|
|1647567213|            3025.0|  2|
|1643201832|            5098.0|  4|
|1644724061|           3265.01| 14|
|1641242797|           1683.99|  5|
|1649173313|1512.8700000000001|  5|
|1650530689|           13850.0|  6|
|1642875447|            9054.0| 12|
|1648041601|            1518.0|  5|
|1648193761|           1231.49|  9|
|1649848613|            9499.0| 20|
|1650981438|            1126.0|  3|
|1645594382|            1363.5| 10|
|1640793161|           13061.5| 14|
|1650515990|            3700.0|  6|
|1649639405|           28839.0| 23|
|1649858595|            1062.5|  7|
|1644343468|            9245.1| 19|
|1640179146|           6666.94| 22|
+----------+------------------+---+
only showi